# Lax-Wendroff method for advection with wave packet initial data

[AMath 586, Spring Quarter 2016](http://faculty.washington.edu/rjl/classes/am586s2016/)

Sample program to solve the advection equation with the Lax-Wendroff method and demonstrate dispersion using wave packet initial data.

In [4]:
%pylab inline

For making animations with an `interact`, we will accumulate a list `figs` of the figures for the animation and then use the function `animate_figs` to loop through them:

In [51]:
from ipywidgets import interact
import ipywidgets as widgets

def animate_figs(figs):
    interact(lambda n: display(figs[n]), n=widgets.IntSlider(min=0,max=len(figs)-1, value=0))

The code from the notebook Lax-Wendroff_periodic is copied below, with a change to the initial data and the ylimits for plotting.

In [43]:
def advection_LW_pbc(m, nsteps, xi0=100, nplot=None, verbose=False):
    """
    Solve u_t + au_x = 0  on [ax,bx] with periodic boundary conditions,
    using the Lax-Wendroff method with m interior points.
     
    Returns k, h, and the max-norm of the error
     
    """

    a = 2             # advection velocity
    
    ax = 0.
    bx = 1.
    tfinal = 1.                # final time
    
    def eta(x):
        """Initial conditions"""
        beta = 100.
        return exp(-beta*(x - 0.5)**2) * cos(xi0*x)

    def utrue(x,t):
        """
        True solution for comparison.
        For periodic BC's, we need the periodic extension of eta(x).
        Map x-a*t-ax back to interval of length bx-ax
        and then evaluate initial data at this point.
        """
        xat = ax + mod(x - a*t - ax, bx-ax)
        return eta(xat)


    h = (bx-ax)/float(m+1)     # h = delta x
    k = tfinal / float(nsteps) # time step
    nu = a*k/h                 # Courant number
    
    xgrid = linspace(ax, bx, m+2)
            
    I = array(range(m+1), dtype=int)   # indices of unknowns 0, 1, ..., m
    Im1 = mod(I-1, m+1)
    Ip1 = mod(I+1, m+1)

    if verbose:
        print "Will take %i time steps with k = %g, h = %g" % (nsteps,k,h)
        if nplot is not None:
            print "    and plot every %i time steps" % nplot

        print "The Courant number is nu = a*k/h = %g" % nu
        if abs(nu) > 1:
                print "*** Warning: the method is unstable!"
                
        print "Wave packet data with xi0 = %g" % xi0
        print "The advection velocity is a = %6.3f" % a
        cg = a - 1/2. * a * h**2 * (1 - (a*k/h)**2) * xi0**2
        print "The group velocity is cg = %6.3f" % cg


    
    # initial conditions:
    u0 = eta(xgrid)

    # initial data on fine grid for plotting:
    xfine = linspace(ax,bx,1001)
    ufine = utrue(xfine,0)

    # make plot directory for storing png files:
    figs = []   # empty list, to accumulate figures
        
    if nplot is not None:
 
        # plot initial data:
        fig = figure(figsize=(10,5))
        plot(xgrid,u0,'b.-', label='computed')
        plot(xfine,ufine,'r', label='true')
        axis([ax,bx,-1.2,1.2])
        legend()
        title('Initial data at time = 0')
    
        # Save this frame:
        figs.append(fig)

        
    # main time-stepping loop:
    
    tn = 0.
    u = eta(xgrid)  # initial data

    for n in range(1,nsteps+1):
        tnp = tn + k   # = t_{n+1}
        
        # Lax-Wendroff:
        u[I] = u[I] - 0.5*nu*(u[Ip1] - u[Im1]) + \
                      0.5*nu**2 * (u[Im1] - 2*u[I] + u[Ip1])
            
        u[m+1] = u[0]  # for plotting

        # plot results at desired times:
        if nplot is not None:
            if mod(n,nplot)==0 or n==nsteps:
                uint = u[0:m+2]  # points on the interval (drop ghost cell on right)
                ufine = utrue(xfine,tnp)
                
                fig = figure(figsize=(10,5))
                plot(xgrid,u,'b.-', label='computed')
                plot(xfine,ufine,'r', label='true')
                axis([ax,bx,-1.2,1.2])
                title('t = %9.5e  after %4i time steps with %5i grid points' \
                               % (tnp,n,m+1))
                # Save this frame:
                figs.append(fig)
                
        tn = tnp   # for next time step
        close('all')  # close figure
        
    error = max(abs(u[0:m+2] - utrue(xgrid,tfinal)))
    return h,k,error,figs


In [60]:
r_vals = array([1,2,4,8,16,32], dtype=int)
m_vals = 50*r_vals - 1
nsteps_vals = 120*r_vals
E = empty(len(nsteps_vals))

# print table header:
print "   h         dt          error      ratio  estimated order"

for j in range(len(r_vals)):
    m = nsteps_vals[j]/4 - 1
    h,dt,E[j],figs = advection_LW_pbc(m=m_vals[j], nsteps=nsteps_vals[j], \
                                      xi0=100)
    if j>0:
        ratio = E[j-1] / E[j]
    else:
        ratio = nan
    p = log(ratio)/log(2)
    print "%8.6f  %8.6f  %12.8f    %4.2f        %4.2f" % (h, dt, E[j], ratio, p)

h_vals = 1./(m_vals + 1)
loglog(h_vals, E, '-o')
title('Log-log plot of errors')
xlabel('nsteps')
ylabel('error')
# savefig('LW_errors.png')

## Create an animation of the solution

In [58]:
h,k,error,figs = advection_LW_pbc(m=299, nsteps=800, \
                                  xi0=100, nplot=20, verbose=True)

In [59]:
animate_figs(figs)